# House Robber III

https://leetcode.com/problems/house-robber-iii/

I am choosing to believe that this is named for a man who is the third first name House, last name Robber in his family

## Initial Thoughts
- This one reminds me a lot of the [binary tree cameras problem](../0968-(hard)-binary-tree-cameras/).
- Like that one, a node cannot decide what state it should be in based on its children alone.
- After much drawing things out on paper, I find this one _much harder_ than the binary tree cameras problem.
- This problem seems to defy my usual approach of yielding solution nodes from the tree. If a node looks at its children and computes the amount that can be stolen by different configurations of its children, it will be traversing the tree many times. This is bad. 

### Approach
Disclaimer - there were a lot of false starts before this.

- Mr. Robber III can either rob a given house or not rob a given house.
- IF he robs a given house, he CANNOT rob its children or its parent
- IF he does not rob a given house, he CAN rob its children and its parents
- Therefore, when thinking about a particular house, if he knows how much he could make by robbing the children, he can determine how much he could possibly get by electing to rob the given house or not rob the given house, but he CANNOT know, without knowing about the parent house, whether robbing the current house will come at the price of a larger payout from robbing the parent house.

Eventually, I cobbled together the following:

```python
# return max for (DO ROB THIS HOUSE, DON'T ROB THIS HOUSE)
def evaluateRobbing(house) -> tuple[int, int]:
    if (house is None):
        return 0, 0

    # If you DO rob this house
    # you gain
    robLeftMax, noRobLeftMax = evaluateRobbing(house.left)

    robRightMax, noRobRightMax = evaluateRobbing(house.right)

    doRobThisHouseMax = house.val + noRobLeftMax + noRobRightMax

    noRobThisHouseMax = robLeftMax + robRightMax

    return doRobThisHouseMax, noRobThisHouseMax
```

But, taking the max of the evaluation at the root failed test cases (perhaps not surprising, since it doesn't even ever use the max function along the way). 

Then it dawned on me - even we don't rob the current house, it may _still_ be better not to rob the child house. So, here's the full, fixed solution:

In [2]:
# return max for (DO ROB THIS HOUSE, DON'T ROB THIS HOUSE)
def evaluateRobbing(house) -> tuple[int, int]:
    if (house is None):
        return 0, 0

    # If you DO rob this house
    # you gain
    robLeftMax, noRobLeftMax = evaluateRobbing(house.left)
    robRightMax, noRobRightMax = evaluateRobbing(house.right)

    doRobThisHouseMax = house.val + noRobLeftMax + noRobRightMax
    noRobThisHouseMax = max(robLeftMax, noRobLeftMax) + max(robRightMax, noRobRightMax)

    return doRobThisHouseMax, noRobThisHouseMax



# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution:
    def rob(self, root) -> int:
        return max(evaluateRobbing(root))
         

## Success
Faster than 38%, less memory than 96%

### Complexity
- Time: `O(n)`
- Space: `O(h)`, with `h` the depth of the tree, and our call stack.

This solution isn't the most elegant. In particular, it is a little tricky to know when we are _definitely_ robbing a house, making my usual generator-based approached more tricky. I think, though, that if we wanted to, we could both return a max and a list of nodes contributing to the max, so that when we pick a path (in the second to last line), we also pass up which decision we made, rather than just the max. I'm not sure how to express that nicely with generators, since we'd need to pass up two candidate iterables, for the parent to make a decision which one to re-yield. Again, probably easier to pass up a concrete list and forgo the lazy iteration.

This one is classified as a medium, but I found it harder than most hards